This notebook will be used as a base and support in the implementation of the nlpaug library for text augmentation.



In [1]:
%load_ext autoreload
%autoreload 2

## Dataset

In [2]:
import pandas as pd

df = pd.read_csv("../../data/silver/df_cleantext_v0.csv")
df

,Category,Message
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in number a wkly comp to win fa cup...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...
...,...,...
5152,1,this is the numbernd time we have tried number...
5153,0,will you b going to esplanade fr home
5154,0,pity was in mood for that soany other suggestions
5155,0,the guy did some bitching but i acted like id ...


In [6]:
import os
import sys
sys.path.append(os.path.abspath("../utils"))
from experiments_utils import print_and_highlight_diff

## Chunk for analysis

In [3]:
pd.set_option('display.max_colwidth', None)

sample = df['Message'].iloc[:5].astype(str).to_list()


In [4]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

## KeyboardAug method

In [6]:
aug = nac.KeyboardAug()
augmented_texts = aug.augment(sample)

print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go ^ntip jurong poib$ crazy available only in bugis n great world la e HuffeR c(je hh4re got ampTe wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar Mok7ng wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry in number a wkly comp to win fa cup fona< tkts nKNberZt may number teDG fa to nImbef to receive en6r& q*wstipnstx txt rxt3tcA app.6 mumbRrovernuJNsgs 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say so ewrlh hor u c SOreWdy Gh@n say 
----------------------

Notes: 
1. The original criteria decided to not focus on special chars nor numbers so the cleaning acted accordingly. These two params should be changed. Same for upper/lower chars. 
2. Many characters of each modified word are changed and the result doesn't look realistic imo. This augmenter technique creates unrealistic results.


In [7]:
aug = nac.KeyboardAug(aug_word_p=0.1, include_numeric=False, include_special_char=False, include_upper_case=False)
augmented_texts = aug.augment(sample)

print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until jirojg point crazy available only in bugis n great world la e buffet done there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar jokibt wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free wgtry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs aoppy nhmberlfefhumberc 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say so ewtly hor u c already tjeg say 
----------------------

Notes: 

The misspellings are very hardcore and artificial. They don't make sense to me. 


In [8]:
# help(nac.KeyboardAug)
# help(aug.augment)

## SpellingAug method

This method substitutes word by spelling mistake words dictionary

In [9]:
aug = naws.SpellingAug()
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
gona utill jurong piont crazy available only il bugis n great would la e buffet cine there got amore want 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
OK. lar joking wife u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
frre entry en unmber e wkly comp to winn fa cup finel tkts numberst may nambr text fa too. number to recived entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
you dun say s orly hor u c already thne say 
---------------

Notes: 

This method creates more realistic results than the previous technique.

In [10]:
aug = naws.SpellingAug(aug_p = 0.5)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go untill jurong point craezy avilable onley jin bugis n grea world a e buffet cinema theve gat amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
OK)][[... lar chocking wife u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
freer entry ith number a wkly comp to win fa couple finel tkts numberst may numper test fa to nuamber ro recidive entry questionstd txt ratetcs apple numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
you dun soy so early hot you c alread thenk say

Notes:

1. The results are way more realistic.
2. The dictionary includes upper case that should be treated later on.    


In [11]:
#help(naws.SpellingAug())
# help(aug.augment)

## SynonymAug

Substitute similar word according to WordNet/ PPDB synonym

Default is WordNet 

In [12]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/maldu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [13]:
aug = naw.SynonymAug()
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until jurong point nutcase available only in bugis n great world la e buffet cine there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar joking wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 31
free entry in number a wkly comp to bring home the bacon fa cup final tkts numberst may number text fa to numeral to get entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 13
u grayish brown say so former hor u one c already t

Notes:

1. Some add extra words and I don't understand why

- Original: 11
- u dun say so early hor u c already then say
- Augmented: 12
- u dun say so other hor u ampere second already then say 

2. Translations look more realistic to me 
3. Checking the params of the function it provides two databases of misspellings 'wordnet' and 'ppdb'.

In [14]:
aug = naw.SynonymAug(aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 23
hold up until jurong head crazy available only in bugis n great world la e buffet cine at that place got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 9
ok lar joking wif u office of naval intelligence 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 30
loose entry in turn a wkly comprehensive examination to win fa cup concluding tkts numberst crataegus laevigata number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Aug

Notes: 

1. Indeed we should be careful with the perc of augmented texts because it adds new words changing the length of the sentence. 
2. Some synonims are not fitting the meaning of the sentence imo.

This cannot be used unless we find a way to force 1-to-1 swapping...not even then. We should also restrict the type of synonym to swap to absolute synonyms :\

In [15]:
# help(naw.SynonymAug())

## WordEmbeddingAug method

This method inserts word randomly by word embeddings similarity. 

:param str model_type: Model type of word embeddings. Expected values include 'word2vec', 'glove' and 'fasttext'.

### Download models




In [17]:
os.makedirs("./models", exist_ok=True)

In [18]:
from nlpaug.util.file.download import DownloadUtil

DownloadUtil.download_word2vec(dest_dir='./models')
DownloadUtil.download_glove('glove.6B', './models')
DownloadUtil.download_fasttext('wiki-news-300d-1M', './models')


Downloading...
From (original): https://drive.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=27125d57-52fa-401b-8a73-de9429426f8e
To: /home/maldu/dscience/projects/spam_detector/research/03_text_augmentation/models/GoogleNews-vectors-negative300.bin.gz
100%|██████████| 1.65G/1.65G [02:27<00:00, 11.2MB/s]


### word2vec


In [19]:

aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='./models/GoogleNews-vectors-negative300.bin',
    action="substitute")
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)


2025-01-03 18:14:20,279 - INFO - loading projection weights from ./models/GoogleNews-vectors-negative300.bin


2025-01-03 18:14:36,889 - INFO - KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from ./models/GoogleNews-vectors-negative300.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-01-03T18:14:36.889408', 'gensim': '4.3.3', 'python': '3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]', 'platform': 'Linux-6.8.0-50-generic-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}


--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
want until jurong Josh_Dotzler crazy available scarcely in bugis n great world en_una e buffet cine there got la_musica micheal 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
Ummmm lar joking gina u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
HEARING_LOSS_OUTREACH entry before number a avail_1Dec mcdonalds to win issa Czech_Republic_Kveta_Peschke final tkts numberst maynot both text fa to geographical_dispersion to receive entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early 

In [20]:
aug = naw.WordEmbsAug(
    model_type='word2vec', model_path='./models/GoogleNews-vectors-negative300.bin',
    action="substitute", aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go before jurong point crazy available only in bugis n great Britain la e buffet cine whatsoever States_Morovich tutti kom 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok ked joking joanna u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
scoliosis_screenings entry over various a prkg nrl to win po cup final tkts numberst doesn_t number text ba to number to giving entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun worried so began 

### glove 6B 100d

In [21]:

aug = naw.WordEmbsAug(
    model_type='glove', model_path='./models/glove.6B.100d.txt',
    action="substitute")
augmented_texts = aug.augment(sample)

print_and_highlight_diff(sample, augmented_texts)

2025-01-03 18:15:09,195 - INFO - loading projection weights from ./models/glove.6B.100d.txt
2025-01-03 18:15:24,849 - INFO - KeyedVectors lifecycle event {'msg': 'loaded (400000, 100) matrix of type float32 from ./models/glove.6B.100d.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-01-03T18:15:24.849573', 'gensim': '4.3.3', 'python': '3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]', 'platform': 'Linux-6.8.0-50-generic-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}


--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
you until jurong much crazy available only in bugis n great world la e buffet d'or things got naqoyqatsi prambanan 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
'd lar joking wif ω oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry in number take wkly comp seek secured fa cup final fukuchiyama numberst end 12 text fa to four to send entry questionstd txt ratetcs able numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u dun say so prior meer u simmons have then say 


In [22]:

aug = naw.WordEmbsAug(
    model_type='glove', model_path='./models/glove.6B.100d.txt',
    action="substitute",  aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until jurong point something dvd only in karanga n great world la blog buffet cine there got loca bhagavathi 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok pra joking wif yang oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry in though a wkly comp to win wembley winner opening whifflet numberst fall least book fa attempt number to receive entry questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u stehf say so early hor je 20 alrea

### fasttext

In [24]:
aug = naw.WordEmbsAug(
    model_type='fasttext', model_path='./models/wiki-news-300d-1M.vec',
    action="substitute")
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

2025-01-03 18:15:42,896 - INFO - loading projection weights from ./models/wiki-news-300d-1M.vec


2025-01-03 18:17:30,377 - INFO - KeyedVectors lifecycle event {'msg': 'loaded (999994, 300) matrix of type float32 from ./models/wiki-news-300d-1M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-01-03T18:17:30.377731', 'gensim': '4.3.3', 'python': '3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]', 'platform': 'Linux-6.8.0-50-generic-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}


--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go until jurong point hysterical available still in bugis n admirable world della e dinners cine they got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok doc- joking yeer u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
open posting playing number a wkly comp to win fa vase 5th tkts numberst may Numbers text fa moving number to garner form questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u needin say so early-to-mid eet u c alread

In [25]:

aug = naw.WordEmbsAug(
    model_type='fasttext', model_path='./models/wiki-news-300d-1M.vec',
    action="substitute", aug_p= 0.3)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
reach until jurong point deranged available only in bugis n HUGE sport la e eating cine anyhow got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lous joking coc u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
zero-cost copy in number over wkly Uni to win fa Vase final tkts numberst may 119 text fa to 89 to distribute attempt questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u owt though --so early soun u c already then say 
----

Notes:

- It's always nice to try but this time we weren't succesful. None of three models made substitutions that would make sense. That is expected since our texts are rather short and there is not much semantic meaning on them. Augmentation based on keywords provided better results.

In [ ]:
# help(naw.WordEmbsAug)

## ContextualWordEmbs method

I'll try SqueezeBERT since the resources are limited 

In [26]:
aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="substitute", aug_p= 0.3)
augmented_text = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
reach until jurong point deranged available only in bugis n HUGE sport la e eating cine anyhow got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lous joking coc u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
zero-cost copy in number over wkly Uni to win fa Vase final tkts numberst may 119 text fa to 89 to distribute attempt questionstd txt ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 11
u owt though --so early soun u c already then say 
----

In [27]:
aug = nawcwe.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', 
    action ='substitute', 
    aug_p= 0.8)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 20
go go jurong went crazy but only in commercials like [UNK] soup [UNK] mama die cine there got amore wat 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 5
rv... ≈ ה [UNK] ≤ 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
free entry in serie a wkly comp web view fa cups final round fixtures may number text fa to fifa 11 receive entry registration request ratetcs apply numberovernumbers 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 10
て 我 [UNK] • early … [UNK] blah... て ！ 
---------------------------

Notes:

As with the word embeddings augmentation method, the results don't make much sense. It also adds simbols even from other languages. That could be restricted by params but after seeing the results I prefer to discard this method 

In [ ]:
# help(nawcwe.ContextualWordEmbsAug)

## BackTranslationAug method

In [9]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en', 
)
augmented_texts = aug.augment(sample)
print_and_highlight_diff(sample, augmented_texts)

--------------------------------------------------
Original: 20
go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Augmented: 33
Go to jurong point crazy available only in bugis n great world la e buffet cine there got amore wat wat a bugis n great world es got amore wat, the buffet c 
--------------------------------------------------
Original: 6
ok lar joking wif u oni
Augmented: 6
ok lar jokes wif u oni 
--------------------------------------------------
Original: 28
free entry in number a wkly comp to win fa cup final tkts numberst may number text fa to number to receive entry questionstd txt ratetcs apply numberovernumbers
Augmented: 28
Free entry to number one wkly comp to win fa cup final tkts numbered numbered can numbered numbered can numbered numbered numbered text fa to numbered fa num 
--------------------------------------------------
Original: 11
u dun say so early hor u c already then say
Augmented: 44
u dun sag so earl

In [8]:
augmented_texts

['Go to jurong point crazy available only in bugis n great world la e buffet cine there got amore wat a bugis amore wat n great world it got amore wat a bugis amore wat n great world it got amore wat it got amore wat, the buffet, the buffet cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine cine it got amore crazy, until crazy, until it crazy, until it crazy, until go to go, until go, until go to go, go, go, go to go, go, go, go, go, go, go, go, go, go up to go, go, go, go, go, go, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go up, go only only only only only only, go, go, go, go up, go, go only only only only only only only go, go, go',
 'ok lar jokes wif u oni',
 'Free entry into number one wkly comp to win cup finals numbered numbered numbered numbered numbered numbered numbered numbered numbered number

Notes:

This method looks interesting but the length of the sentences is going out of control. 

EDIT: the is no way to force the same length in every sentence without modifying the augment class so this method is discarded.